## Implementation of Hybrid ESN for Lorenz 96

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV


### Data Generation and Noising

### Ensemble Kalman Filtering

In [ ]:
class EnKF:
    def __init__(self, ensemble, forecast_func, H, R):
        """
        EnKF implementation based on a forecast model.

        Parameters:
        - ensemble: numpy array of shape (N_ens, state_dim)
        - forecast_func: function to forecast next state (applied to each ensemble member)
        - H: observation matrix (obs_dim x state_dim)
        - R: observation noise covariance (obs_dim x obs_dim)
        """
        self.ensemble = ensemble
        self.forecast_func = forecast_func
        self.H = H
        self.R = R

    def predict(self):
        for i in range(len(self.ensemble)):
            self.ensemble[i] = self.forecast_func(self.ensemble[i])

    def update(self, observation):
        N = len(self.ensemble)
        X = np.array(self.ensemble).T  # shape: (state_dim, N)
        x_mean = np.mean(X, axis=1, keepdims=True)
        X_prime = X - x_mean

        # Observation space
        HX = self.H @ X
        y_mean = np.mean(HX, axis=1, keepdims=True)
        Y_prime = HX - y_mean

        # Kalman gain
        P_xy = X_prime @ Y_prime.T / (N - 1)
        P_yy = Y_prime @ Y_prime.T / (N - 1) + self.R
        K = P_xy @ np.linalg.inv(P_yy)

        # Update ensemble
        for i in range(N):
            perturb = np.random.multivariate_normal(np.zeros(self.R.shape[0]), self.R)
            innovation = observation + perturb - HX[:, i]
            X[:, i] = X[:, i] + K @ innovation

        self.ensemble = X.T

### Hybrid ESN

### Evaluation Metrics